In [111]:
"""
#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings
"""

'\n#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings\n'

In [2]:
import pandas as pd
import matplotlib

# switch to suppress CSV writing (which is slow)
write_csv = True

In [3]:
# BRIT UT colab
df_occ = pd.read_csv('torch_bels_BRIT_UT_locs.csv', low_memory=False, sep='\t')


In [17]:
# file name components to make writing CSVs dynamic and prevent overwrites
batch_prefix = 'TORCH'
batch_designator = 'BRIT_UT'

In [7]:
df_occ.shape

(513076, 96)

In [8]:
# Counties for initial comparision
tx_transpecos_counties = ['Brewster','Jeff Davis','Presidio','El Paso','Val Verde','Culberson','Hudspeth','Terrell','Reeves','Winkler','Ward','Crockett','Loving','Crane','Upton']

In [32]:
# Drop irrelevant colums
# NOTE not all of these columns occur across all collections
#drop_columns= ['higherClassification','kingdom','phylum','class','order','identificationReferences','identificationRemarks','taxonRemarks','identificationQualifier','typeStatus','fieldNumber','eventID','informationWithheld','dataGeneralizations','dynamicProperties','associatedSequences','associatedTaxa','reproductiveCondition','establishmentMeans','lifeStage','sex','individualCount','samplingProtocol','preparations','continent','waterBody','islandGroup','island','rights','rightsHolder','accessRights','recordID','type','license','bibliographicCitation','datasetName','fieldNotes','countryCode','nomenclaturalCode','nomenclaturalStatus','associatedMedia','higherGeography','institutionID','georeferencedDate','datasetID','occurrenceStatus','verbatimLocality','organismID','previousIdentifications','eventTime','eventRemarks','locationAccordingTo','verbatimCoordinateSystem','footprintWKT','earliestEonOrLowestEonothem','earliestEraOrLowestErathem','earliestPeriodOrLowestSystem','earliestEpochOrLowestSeries','earliestAgeOrLowestStage','group','formation','member','identificationVerificationStatus','scientificNameID']

# more agressive drop
drop_columns= ['higherClassification','kingdom','phylum','class','order','identificationReferences',
               'identificationRemarks','taxonRemarks','identificationQualifier','typeStatus','fieldNumber',
               'eventID','informationWithheld','dataGeneralizations','dynamicProperties','associatedSequences',
               'associatedTaxa','reproductiveCondition','establishmentMeans','lifeStage','sex','individualCount',
               'samplingProtocol','preparations','continent','waterBody','islandGroup','island','rights','rightsHolder',
               'accessRights','recordID','type','license','bibliographicCitation','datasetName','fieldNotes','countryCode',
               'nomenclaturalCode','nomenclaturalStatus','associatedMedia','higherGeography','institutionID','georeferencedDate',
               'datasetID','occurrenceStatus','verbatimLocality','organismID','previousIdentifications','eventTime',
               'eventRemarks','locationAccordingTo','verbatimCoordinateSystem','footprintWKT','earliestEonOrLowestEonothem',
               'earliestEraOrLowestErathem','earliestPeriodOrLowestSystem','earliestEpochOrLowestSeries','earliestAgeOrLowestStage',
               'group','formation','member','identificationVerificationStatus','scientificNameID',
               'basisOfRecord','subgenus','specificEpithet','verbatimTaxonRank','infraspecificEpithet','taxonRank','identifiedBy',
               'dateIdentified','year','month','day','startDayOfYear','endDayOfYear'
              ]


df_occ = df_occ.drop(columns=drop_columns, errors='ignore')

In [33]:
df_occ.shape

(513076, 49)

In [34]:
# Find duplicate records
df_matches = df_occ[df_occ['bels_location_string'].duplicated(keep=False)]
#df_matches = df_denton[df_denton.bels_matchwithcoords

In [35]:
df_matches.shape

(327700, 49)

In [36]:
# Add location ID
# first create a copy to avoid copy of a slice of a DF
df_matches = df_matches.copy()
# https://stackoverflow.com/a/51110197 or https://stackoverflow.com/a/51110205
df_matches['loc_id'] = df_matches.groupby(['bels_location_string']).ngroup()

In [37]:
# Add dup loc count column for the size of each location cluster (number of duplicate locations)
#https://stackoverflow.com/a/46768694
df_matches['dup_loc_count'] = df_matches.groupby(['bels_location_string']).transform('size')

In [39]:
# Add a count of the number of records in each dup gruoup that have a geocoord
# https://www.statology.org/pandas-groupby-count-with-condition/
#df.groupby('var1')['var2'].apply(lambda x: (x=='val').sum()).reset_index(name='count')
#['decimalLatitude'].isna()
df_counts = df_matches.groupby('loc_id')['decimalLatitude'].apply(lambda x: (x.isna()).sum()).reset_index(name='count')

In [40]:
# With help from Claude.ai, a new approach to store the results into the original dataframe:
df_matches['loc_geo_count'] = df_matches.groupby('loc_id')['decimalLatitude'].transform(lambda x: (x.notna()).sum())

In [41]:
df_matches.shape

(327700, 52)

In [42]:
# count number of locations with no georef
df_zero_geo = df_matches[df_matches['loc_geo_count'] == 0]

In [47]:
df_zero_geo.shape

(153264, 52)

In [50]:
# number of unique locations with zero geo
#df_zero_geo['loc_id'].nunique()

36338

In [53]:
# in Texas
df_zero_geo_tx = df_zero_geo[df_zero_geo['stateProvince'] == 'Texas']

In [54]:
df_zero_geo_tx.shape

(128698, 52)

In [44]:
if write_csv:
    filename = batch_prefix+'_'+batch_designator+'_matches_loc_id.csv'
    df_matches.to_csv(filename)

In [45]:
if write_csv:
    filename = batch_prefix+'_'+batch_designator+'_zero_geo.csv'
    df_zero_geo.to_csv(filename)

In [60]:
# filter loc with insufficient locality string
# Drop records with only the state name in bels string
df_zero_geo_state_only = df_zero_geo_tx[~df_zero_geo_tx.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]

# remove state + county matches
df_zero_geo_state_county_only = df_zero_geo_state_only[~df_zero_geo_state_only.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]



In [61]:
df_zero_geo_state_county_only.shape

(124596, 52)

In [62]:
if write_csv:
    filename = batch_prefix+'_'+batch_designator+'_zero_geo_state_county_only.csv'
    #df_matches.to_csv('TORCH_bels_matches_loc_id.csv')
    df_zero_geo_state_county_only.to_csv(filename)

In [ ]:
# TODO
# remove other strings that indicate low likelyhood of being able to georef
#texasnoadditionallocalitydataonsheet
#texastarrantnodata
#texasdentondentonconoadditionallocalitydataonsheet
#texashendersonnoadditionallocalitydataonsheet
# texashendersonhendersonconoadditionallocalitydataonsheet





In [78]:
# add counts by county
#df_nogeo_with_geolocdups_nostatecounty
county_summary = df_zero_geo_state_county_only.groupby(['stateProvince','county']).agg(
    total_location_recs=('loc_id', 'count'),
    #sum_id_count=('dup_loc_count', 'sum'),
    unique_locations=('loc_id', 'nunique'),
    #dup_loc_count=('dup_loc_count', 'min'),
    loc_geo_count=('loc_geo_count', 'max')
)

In [79]:
county_summary

total_location_recs  unique_locations  loc_geo_count
stateProvince county                                                        
Texas         Anderson                 1445               265              0
              Andrews                   146                44              0
              Angelina                  715               166              0
              Aransas                   941               207              0
              Archer                    232                43              0
...                                     ...               ...            ...
              Yoakum                     18                 5              0
              Young                     137                42              0
              Zapata                    246                72              0
              Zavala                    163                45              0
              mitchell                    1                 1              0

[264 rows x 3 columns]

In [81]:
if write_csv:
    filename = batch_prefix+'_'+batch_designator+'_county_summary.csv'
    #df_matches.to_csv('TORCH_bels_matches_loc_id.csv')
    county_summary.to_csv(filename)

In [88]:
# add counts by location
loc_summary = df_zero_geo_state_county_only.groupby(['stateProvince','county', 'loc_id']).agg(
    dup_loc_count=('loc_id', 'count'),
    #sum_id_count=('dup_loc_count', 'sum'),
    #dup_loc_count=('dup_loc_count', 'max'),
    #loc_geo_count=('loc_geo_count', 'max')
)

In [89]:
loc_summary

dup_loc_count
stateProvince county   loc_id               
Texas         Anderson 6811                4
                       6812                2
                       6813                4
                       6814                2
                       6815                4
...                                      ...
              Zavala   65426               3
                       65427               2
                       65428               2
                       65429               3
              mitchell 44056               1

[30607 rows x 1 columns]

In [86]:
if write_csv:
    filename = batch_prefix+'_'+batch_designator+'_loc_summary.csv'
    #df_matches.to_csv('TORCH_bels_matches_loc_id.csv')
    loc_summary.to_csv(filename)

In [125]:
# find records lacking geo
#bels_decimallatitude
df_nogeo = df_matches[df_matches['decimalLatitude'].isna()]
#TODO add loc ID - https://stackoverflow.com/a/51110205


In [126]:
df_nogeo.shape

(208011, 64)

In [127]:
if write_csv:
    df_nogeo.to_csv('TORCH_bels_matches_nogeo.csv')

In [128]:
# find records with geo
df_geo = df_matches[df_matches['decimalLatitude'].notna()]

In [131]:
# all dups with and without geocoords
dup_loc_count = df_matches.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [132]:
dup_loc_count.shape

(65430,)

In [133]:
if write_csv:
    dup_loc_count.to_csv('TORCH_dup_loc_count.csv')

In [134]:
# dups with no goecoords
#dups = df.pivot_table(index = ['Course'], aggfunc ='size') 
dup_loc_count_no_geo = df_nogeo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [135]:
dup_loc_count_no_geo.shape

(44769,)

In [136]:
if write_csv:
    dup_loc_count_no_geo.to_csv('TORCH_dup_loc_count_nogeo.csv')

In [137]:
# dups with goecoords
dup_loc_count_geo = df_geo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [138]:
dup_loc_count_geo.shape

(29092,)

In [139]:
if write_csv:
    dup_loc_count_geo.to_csv('TORCH_dup_loc_count_geo.csv')

In [148]:
#TODO
#Load ids of elite georeferencers - EG
df_georeferencers = pd.read_csv('TORCH-georeferencers_tested.csv', low_memory=False)

#find records with geo that have been done by EGs
#find records without geo that match those done by EGs

In [149]:
# filter records that have been georeferenced by 'elite' vetted georeferencers
df_by_egeo = df_matches[df_matches['georeferencedBy'].isin(df_georeferencers['Username'])]

In [150]:
df_by_egeo.shape

(30372, 64)

In [151]:
if write_csv:
    df_by_egeo.to_csv('TORCH_df_by_egeo.csv')

In [152]:
# find locations that match loc_id of those done by EG
df_loc_match_by_egeo = df_matches[df_matches['loc_id'].isin(df_by_egeo['loc_id'])]

In [153]:
df_loc_match_by_egeo.shape

(45211, 64)

In [154]:
# find matching locs that have not been georeffed
df_loc_match_by_egeo_nogeo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].isna()]

In [155]:
df_loc_match_by_egeo_nogeo.shape

(9424, 64)

In [156]:
# find matching locs that have been georeffed
df_loc_match_by_egeo_w_geo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].notna()]

In [157]:
df_loc_match_by_egeo_w_geo.shape

(35787, 64)

In [158]:
df_nogeo.shape

(208011, 64)

In [159]:
# find dup locs without geo but have other dups that have geo
df_nogeo_with_geolocdups = df_nogeo[df_nogeo['loc_id'].isin(df_geo['loc_id'])]

In [160]:
df_nogeo_with_geolocdups.shape

(54747, 64)

In [161]:
if write_csv:
    df_nogeo_with_geolocdups.to_csv('TORCH_nogeo_with_geolocdups.csv')

In [162]:
#TODO 



In [163]:
# Drop records with only the state name in bels string
#df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[df_nogeo_with_geolocdups[column1] != df[column2]]

df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[~df_nogeo_with_geolocdups.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]


In [164]:
df_nogeo_with_geolocdups_nostates.shape

(54747, 64)

In [165]:
if write_csv:
    df_nogeo_with_geolocdups_nostates.to_csv('TORCH_nogeo_with_geolocdups_nostates.csv')

In [166]:
# remove state + county matches
df_nogeo_with_geolocdups_nostatecounty = df_nogeo_with_geolocdups_nostates[~df_nogeo_with_geolocdups_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]

In [167]:
df_nogeo_with_geolocdups_nostatecounty.shape

(54669, 64)

In [168]:
if write_csv:
    df_nogeo_with_geolocdups_nostatecounty.to_csv('TORCH_nogeo_with_geolocdups_nostatecounties.csv')

In [169]:
list[df_nogeo_with_geolocdups_nostatecounty.columns]

list[Index(['id', 'institutionCode', 'collectionCode', 'ownerInstitutionCode',
       'collectionID', 'basisOfRecord', 'occurrenceID', 'catalogNumber',
       'otherCatalogNumbers', 'family', 'scientificName', 'taxonID',
       'scientificNameAuthorship', 'genus', 'subgenus', 'specificEpithet',
       'verbatimTaxonRank', 'infraspecificEpithet', 'taxonRank',
       'identifiedBy', 'dateIdentified', 'recordedBy', 'recordNumber',
       'eventDate', 'year', 'month', 'day', 'startDayOfYear', 'endDayOfYear',
       'verbatimEventDate', 'occurrenceRemarks', 'habitat',
       'associatedOccurrences', 'locationID', 'country', 'stateProvince',
       'county', 'municipality', 'locality', 'locationRemarks',
       'decimalLatitude', 'decimalLongitude', 'geodeticDatum',
       'coordinateUncertaintyInMeters', 'verbatimCoordinates',
       'georeferencedBy', 'georeferenceProtocol', 'georeferenceSources',
       'georeferenceVerificationStatus', 'georeferenceRemarks',
       'minimumElevationInMet

In [188]:
# add counts by county
#df_nogeo_with_geolocdups_nostatecounty
county_summary = df_nogeo_with_geolocdups_nostatecounty.groupby(['stateProvince','county']).agg(
    total_locations=('loc_id', 'count'),
    #sum_id_count=('dup_loc_count', 'sum'),
    unique_locations=('loc_id', 'nunique'),
    dup_loc_count=('dup_loc_count', 'max')
)

In [189]:
county_summary

total_locations  unique_locations  dup_loc_count
stateProvince county                                                   
Oklahoma      Adair                144                29             54
              Alfalfa               11                 6             14
              Atoka                 55                12             23
              Beaver                 7                 4              5
              Beckham                5                 2              5
...                                ...               ...            ...
Texas         Wood                 220                31             44
              Yoakum                 5                 2              9
              Young                 14                 8             47
              Zapata               193                89             65
              Zavala                 2                 2              4

[326 rows x 3 columns]

In [172]:
county_summary.to_csv('TORCH_county_summary.csv')

In [173]:
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# remove states only
df_nogeo_nostates = df_nogeo[~df_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_nogeo_nostatecounty = df_nogeo_nostates[~df_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]

df_nogeo_nostatecounty.shape


(202546, 64)

In [174]:
county_summary_nogeo = df_nogeo_nostatecounty.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count')
)
# Reset the index to make 'state' and 'county' regular columns
county_summary_nogeo = county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(county_summary_nogeo)


Summary with reset index:
    stateProvince    county  unique_locations  loc_count
0        Oklahoma     Adair               113        433
1        Oklahoma   Alfalfa                29         80
2        Oklahoma  Angelina                 1          2
3        Oklahoma     Atoka               168        857
4        Oklahoma    Beaver                30        147
..            ...       ...               ...        ...
345         Texas    Yoakum                 7         23
346         Texas     Young                50        151
347         Texas    Zapata               161        439
348         Texas    Zavala                47        165
349         Texas  mitchell                 1          1

[350 rows x 4 columns]


In [175]:
# Same process but only include BRIT records in Texas
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# Texas collections only
df_torch_texas_nogeo = df_nogeo[df_nogeo['stateProvince'] == 'Texas']
# remove states only
df_torch_texas_nogeo_nostates = df_torch_texas_nogeo[~df_torch_texas_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_torch_texas_nogeo_nostatecounty = df_torch_texas_nogeo_nostates[~df_torch_texas_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]


In [176]:
df_torch_texas_nogeo_nostatecounty.shape  # old (106414, 64)

(175474, 64)

In [177]:
# BRIT collections only
df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_torch_texas_brit_nogeo.shape


(72524, 64)

In [178]:
df_torch_texas_brit_nogeo.to_csv('torch_texas_brit_nogeo.csv')

In [179]:
# Summarize BRIT Texas records
brit_texas_county_summary_nogeo = df_torch_texas_brit_nogeo.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count'),
    largest_dups = ('dup_loc_count', 'max')
)
# Reset the index to make 'state' and 'county' regular columns
brit_texas_county_summary_nogeo = brit_texas_county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(brit_texas_county_summary_nogeo)


Summary with reset index:
    stateProvince    county  unique_locations  loc_count  largest_dups
0           Texas  Anderson               204        809            77
1           Texas   Andrews                 3          4             4
2           Texas  Angelina               136        421            70
3           Texas   Aransas               175        725           249
4           Texas    Archer                38        145            44
..            ...       ...               ...        ...           ...
225         Texas      Wise                79        221            68
226         Texas      Wood               185        882            58
227         Texas     Young                35         99            47
228         Texas    Zapata               133        337            65
229         Texas    Zavala                28         85            16

[230 rows x 5 columns]


In [180]:
#df_torch_texas_nogeo_nostatecounty.shape
# export BRIT Texas summary
brit_texas_county_summary_nogeo.to_csv('TORCH_brit_texas_county_summary_nogeo.csv')

In [181]:
#county_summary_nogeo.to_csv('TORCH_county_summary_nogeo.csv')

In [182]:
# Sampling individual counties
#df_torch_texas_brit_nogeo
# df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_brit_young_nogeo = df_torch_texas_brit_nogeo[df_torch_texas_brit_nogeo['county'] == 'Young']

df_brit_young_nogeo.shape



(99, 64)

In [183]:
df_brit_young_nogeo.to_csv('brit_young_nogeo.csv')